## Sprawozdanie1

**Data:** 2024-06-09  
**Autor:** Piotr Szepietowski

## Metoda iteracyjna Jacobiego

Metoda Jacobiego jest iteracyjną metodą rozwiązywania układów równań liniowych. Polega na następujących krokach:

1. **Warunek początkowy:** Układ równań musi być zapisany w postaci macierzowej Ax = b, gdzie:
    - A jest macierzą współczynników
    - x jest wektorem niewiadomych
    - b jest wektorem wyrazów wolnych

2. **Przekształcenie układu:** Dla każdego równania wyraża się niewiadomą xi przez pozostałe zmienne:
    ```
    x₁ = (b₁ - a₁₂x₂ - a₁₃x₃ - ... - a₁ₙxₙ) / a₁₁
    x₂ = (b₂ - a₂₁x₁ - a₂₃x₃ - ... - a₂ₙxₙ) / a₂₂
    ...
    xₙ = (bₙ - aₙ₁x₁ - aₙ₂x₂ - ... - aₙ,ₙ₋₁xₙ₋₁) / aₙₙ
    ```

3. **Iteracja:** W każdej iteracji k+1 nowe wartości x obliczane są na podstawie wartości z poprzedniej iteracji k:
    ```
    x⁽ᵏ⁺¹⁾ᵢ = (bᵢ - Σⱼ₌₁,ⱼ≠ᵢ aᵢⱼx⁽ᵏ⁾ⱼ) / aᵢᵢ
    ```

4. **Zbieżność:** Metoda jest zbieżna, gdy:
    - Macierz A jest diagonalnie dominująca
    - Elementy na przekątnej głównej są niezerowe

5. **Kryterium stopu:** Iteracje są wykonywane do momentu osiągnięcia zadanej dokładności ε:
    ```
    ||x⁽ᵏ⁺¹⁾ - x⁽ᵏ⁾|| < ε
    ```
---
## Metoda iteracyjna Gaussa-Seidla

Metoda Gaussa-Seidla jest udoskonaloną wersją metody Jacobiego, w której do obliczania nowych wartości x wykorzystuje się już zaktualizowane wartości z bieżącej iteracji.

1. **Warunek początkowy:** Układ równań w postaci Ax = b.

2. **Przekształcenie układu:** Podobnie jak w metodzie Jacobiego, każdą niewiadomą wyraża się przez pozostałe zmienne.

3. **Iteracja:** Nowe wartości x obliczane są sekwencyjnie, natychmiast wykorzystując najnowsze dostępne wartości:
    ```
    x⁽ᵏ⁺¹⁾ᵢ = (bᵢ - Σⱼ₌₁ⁱ₋₁ aᵢⱼx⁽ᵏ⁺¹⁾ⱼ - Σⱼ₌ᵢ₊₁ⁿ aᵢⱼx⁽ᵏ⁾ⱼ) / aᵢᵢ
    ```

4. **Zbieżność:** Metoda jest zbieżna, gdy macierz A jest diagonalnie dominująca lub spełnia warunek zbieżności.

5. **Kryterium stopu:** Iteracje są wykonywane do osiągnięcia zadanej dokładności ε:
    ```
    ||x⁽ᵏ⁺¹⁾ - x⁽ᵏ⁾|| < ε
    ```

---

## Metoda iteracyjna SOR (Successive Over-Relaxation)

Metoda SOR jest rozszerzeniem metody Gaussa-Seidla, w której wprowadza się parametr relaksacji ω (omega), aby przyspieszyć zbieżność.

1. **Warunek początkowy:** Układ równań w postaci Ax = b.

2. **Przekształcenie układu:** Każdą niewiadomą wyraża się przez pozostałe zmienne, jak w poprzednich metodach.

3. **Iteracja:** Nowe wartości x obliczane są z użyciem parametru relaksacji ω:
    ```
    x⁽ᵏ⁺¹⁾ᵢ = (1 - ω)x⁽ᵏ⁾ᵢ + (ω / aᵢᵢ) * (bᵢ - Σⱼ₌₁ⁱ₋₁ aᵢⱼx⁽ᵏ⁺¹⁾ⱼ - Σⱼ₌ᵢ₊₁ⁿ aᵢⱼx⁽ᵏ⁾ⱼ)
    ```
    gdzie 0 < ω < 2.

4. **Zbieżność:** Wybór odpowiedniego ω (najczęściej 1 < ω < 2) może znacząco przyspieszyć zbieżność metody.

5. **Kryterium stopu:** Iteracje są wykonywane do osiągnięcia zadanej dokładności ε:
    ```
    ||x⁽ᵏ⁺¹⁾ - x⁽ᵏ⁾|| < ε
    ```

In [ ]:
import numpy as np
import time
from functools import wraps

# Dictionaries to store times and iteration counts for each method and matrix size
timings = {'jacobi': [], 'gauss_seidel': [], 'sor': []}
iterations = {'jacobi': [], 'gauss_seidel': [], 'sor': []}
sizes = []

def record_stats(method_name, size, elapsed, iters):
    timings[method_name].append(elapsed)
    iterations[method_name].append(iters)
    if method_name == 'jacobi':  # Only append size once per set
        sizes.append(size)

def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result, iter_count, printExtended = func(*args, **kwargs)
        end_time = time.time()
        elapsed = end_time - start_time
        method_name = func.__name__
        A = args[0]
        size = A.shape[0]
        record_stats(method_name, size, elapsed, iter_count + 1 if method_name == 'jacobi' else iter_count)
        if printExtended:
            print(f"{method_name} took {elapsed:.4f} seconds to execute")
            print(f"{method_name} solution: {result}")
            print(f"Number of iterations: {iter_count}\n")
        return result
    return wrapper

@timer
def jacobi(A, b, x0=None, tol=1e-10, printExtended=True):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()
    D = np.diag(A)
    R = A - np.diagflat(D)
    iter_count = 0
    while True:
        x_new = (b - np.dot(R, x)) / D
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, iter_count + 1, printExtended
        x = x_new
        iter_count += 1

@timer
def gauss_seidel(A, b, x0=None, tol=1e-10, printExtended=True):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()
    iter_count = 0
    while True:
        x_new = x.copy()
        for i in range(n):
            sum1 = np.dot(A[i, :i], x_new[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - sum1 - sum2) / A[i, i]
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, iter_count + 1, printExtended
        x = x_new
        iter_count += 1

@timer
def sor(A, b, x0=None, omega=1.0, tol=1e-10, printExtended=True):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()
    iter_count = 0
    while True:
        x_new = x.copy()
        for i in range(n):
            sum1 = np.dot(A[i, :i], x_new[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (1 - omega) * x[i] + (omega * (b[i] - sum1 - sum2)) / A[i, i]
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, iter_count + 1, printExtended
        x = x_new
        iter_count += 1

## Test na przykładzie zawartym w prezentacji
Porównanie wyników oraz czasów i liczby koniecznych iteracji dla wszystkich trzech metod przy podanym w prezentacji przykładzie:

**Macierz A:**
$$
A = \begin{bmatrix}
3 & -1 & 0 & 0 & 0 & -1 \\
-1 & 3 & -1 & 0 & -1 & 0 \\
0 & -1 & 3 & -1 & 0 & 0 \\
0 & 0 & -1 & 3 & -1 & 0 \\
0 & -1 & 0 & -1 & 3 & -1 \\
-1 & 0 & 0 & 0 & -1 & 3 \\
\end{bmatrix}
$$

**Wektor b:**
$$
b = \begin{bmatrix}
0 \\
0 \\
0 \\
0 \\
0 \\
20 \\
\end{bmatrix}
$$

Poniżej przedstawiono wyniki, czasy wykonania oraz liczbę iteracji dla metod Jacobiego, Gaussa-Seidla i SOR.

In [ ]:
A = np.array([[3,-1,0,0,0,-1], 
                  [-1,3,-1,0,-1,0],
                  [0,-1,3,-1,0,0],
                  [0,0,-1,3,-1,0],
                  [0,-1,0,-1,3,-1],
                  [-1,0,0,0,-1,3]], dtype=float)
b = np.array([0, 0, 0, 0, 0, 20], dtype=float)
print("Solving Ax = b using Jacobi, Gauss-Seidel, and SOR methods:\n")
print(f"Matrix A: {A}")
print(f"Vector b: {b}\n")
jacobi(A, b)
gauss_seidel(A, b)
sor(A, b, omega=1.25)

In [ ]:
np.random.seed(42)
n_big = 100
A_big = np.random.rand(n_big, n_big)
for i in range(n_big):
    A_big[i, i] = np.sum(np.abs(A_big[i])) + 1
b_big = np.random.rand(n_big)
print("Solving large Ax = b using Jacobi, Gauss-Seidel, and SOR methods:\n")
print(f"Matrix A_big shape: {A_big.shape}")
print(f"Vector b_big shape: {b_big.shape}\n")
jacobi(A_big, b_big)
gauss_seidel(A_big, b_big)
sor(A_big, b_big, omega=1.1)

In [ ]:
for n_big in range(200, 2001, 100):
    np.random.seed(42)
    A_big = np.random.rand(n_big, n_big)
    for i in range(n_big):
        A_big[i, i] = np.sum(np.abs(A_big[i])) + 1
    b_big = np.random.rand(n_big)
    jacobi(A_big, b_big, printExtended=False)
    gauss_seidel(A_big, b_big, printExtended=False)
    sor(A_big, b_big, omega=1.1, printExtended=False)

for n_big in range(2200, 4001, 200):
    np.random.seed(42)
    A_big = np.random.rand(n_big, n_big)
    for i in range(n_big):
        A_big[i, i] = np.sum(np.abs(A_big[i])) + 1
    b_big = np.random.rand(n_big)
    jacobi(A_big, b_big, printExtended=False)
    gauss_seidel(A_big, b_big, printExtended=False)
    sor(A_big, b_big, omega=1.1, printExtended=False)

In [ ]:
import matplotlib.pyplot as plt

# Plot timings
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
for method in timings:
    plt.plot(sizes, timings[method], marker='o', label=method.capitalize())
plt.xlabel('Matrix size (n)')
plt.ylabel('Time (seconds)')
plt.title('Execution Time vs Matrix Size')
plt.legend()
plt.grid(True)

# Plot iterations
plt.subplot(1, 2, 2)
for method in iterations:
    plt.plot(sizes, iterations[method], marker='o', label=method.capitalize())
plt.xlabel('Matrix size (n)')
plt.ylabel('Number of Iterations')
plt.title('Iterations vs Matrix Size')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()